# Magnetized Spheres

This code calculates some magnetical problems with a spherical simetry.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from fatiando import mesher, gridder, utils
from fatiando.gravmag import sphere
from fatiando.mesher import Sphere
from fatiando.vis import mpl
from fatiando.constants import CM, T2NT
from matplotlib import colors, ticker, cm
import mag_sphere_functions as mfun
% matplotlib inline

## Spheres

In [ ]:
inc, dec = -30, 50 # inclination and declination of regional field

# creating the list of spheres
props1 = {'magnetization':utils.ang2vec(5, inc, dec)} # defining induced magnetization
props2 = {'magnetization': utils.ang2vec(10, 70, -50)} # defining remanent magnetization
model = [
    [10000, 12000, 600, 50,props1],
    [10000, 20000, 600, 50,props1]] # list of sphere models [x0,y0,z0,R,props]

# model for comparision with fatiando
model_fat = [
    Sphere(10000, 12000, 600, 50,
          {'magnetization': utils.ang2vec(5, inc, dec)}),
    Sphere(10000, 20000, 600, 50,
          {'magnetization': utils.ang2vec(5, inc, dec)})] # input fatiando

## Observation Points

In [ ]:
#area over which the data are calculated
#x minimum, x maximum, y minimum and y maximum
area = [0, 30000, 5000, 35000] 

#number of data along the y and x directions
shape = (100,100)

#total number of data
N = shape[0]*shape[1]

#coordinates x and y of the data
x = np.linspace(area[0],area[1],shape[0]) # points in x
y = np.linspace(area[2],area[3],shape[0]) # points in y
xp,yp = np.meshgrid(x,y)    # creating mesh points
xp = xp.ravel()
yp = yp.ravel()

#vertical coordinates of the data
zp = -350. - 500.*utils.gaussian2d(xp, yp, 17000, 21000, 21000, 18500, angle=21)

In [ ]:
mpl.figure(figsize=(10,10))
mpl.title("Relief (m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, zp,shape,30, cmap=mpl.cm.Greens_r)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.colorbar(pad=0.01, aspect=20, shrink=1.0).set_label('nT')
mpl.m2km()
mpl.show()

### Result for magnetic induction of my function

In [ ]:
Bx = mfun.bx_sphere(xp,yp,zp,model)
By = mfun.by_sphere(xp,yp,zp,model)
Bz = mfun.bz_sphere(xp,yp,zp,model)

### Result for magnetic induction from fatiando

In [ ]:
bx_fat = sphere.bx(xp,yp,zp,model_fat)
by_fat = sphere.by(xp,yp,zp,model_fat)
bz_fat = sphere.bz(xp,yp,zp,model_fat)

### Comparison with fatiando

In [ ]:
np.allclose(Bx,bx_fat)

In [ ]:
np.allclose(By,by_fat)

In [ ]:
np.allclose(Bz,bz_fat)

### Ploting results

In [ ]:
mpl.figure(figsize=(15,15))

mpl.subplot(2,3,1)
mpl.axis('scaled')
mpl.title("Bx (nT)")
mpl.contourf(yp, xp, Bx,shape, 15)
mpl.xlabel('East y (m)')
mpl.ylabel('North x (m)')
mpl.m2km()

mpl.subplot(2,3,2)
mpl.axis('scaled')
mpl.title("By (nT)")
mpl.contourf(yp, xp, By,shape, 15)
mpl.xlabel('East y (m)')
mpl.ylabel('North x (m)')
mpl.m2km()

mpl.subplot(2,3,3)
mpl.axis('scaled')
mpl.title("Bz (nT)")
mpl.contourf(yp, xp, Bz,shape, 15)
mpl.xlabel('East y (m)')
mpl.ylabel('North x (m)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.5).set_label('nT')


mpl.subplot(2,3,4)
mpl.axis('scaled')
mpl.title("Bx_fat (nT)")
mpl.contourf(yp, xp, bx_fat,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()

mpl.subplot(2,3,5)
mpl.axis('scaled')
mpl.title("By_fat (nT)")
mpl.contourf(yp, xp, by_fat,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()

mpl.subplot(2,3,6)
mpl.axis('scaled')
mpl.title("Bz_fat (nT)")
mpl.contourf(yp, xp, bz_fat,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.5).set_label('nT')
mpl.show()

### Result for total field anomaly of my function

In [ ]:
tf = mfun.tf_sphere(xp,yp,zp,model,inc,dec)

### Result for total field anomaly from fatiando

In [ ]:
tf_fat = sphere.tf(xp,yp,zp,model_fat,inc,dec)

### Comparison with fatiando

In [ ]:
np.allclose(tf,tf_fat)

### Ploting results

In [ ]:
mpl.figure(figsize=(10,10))
mpl.axis('scaled')

mpl.subplot(1,2,1)
mpl.axis('scaled')
mpl.title("TFA (nT)")
mpl.contourf(yp, xp, tf,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()

mpl.subplot(1,2,2)
mpl.axis('scaled')
mpl.title("TFA_fat (nT)")
mpl.contourf(yp, xp, tf_fat,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.5).set_label('nT')
mpl.show()

### Creating a layer with shperes

In [ ]:
# creating a list of shperes below each observation points
model_layer = []
h = 100.                   # layer height
for i in range(len(xp)):
    model_layer.append([xp[i],yp[i],h,1.,props1])
    
    
# creating a list of shperes below each observation points with remanent
model_layer_remanent = []
h = 100.                   # layer height
for i in range(len(xp)):
    model_layer_remanent.append([xp[i],yp[i],h,1.,props2])

### Result for foward problem

In [ ]:
A,vp = mfun.sm_sphere(xp,yp,zp,model_layer,inc,dec)

In [ ]:
tf_layer_fd = np.dot(A,vp)

### Comparison with TFA function

In [ ]:
tf_layer = mfun.tf_sphere(xp,yp,zp,model_layer,inc,dec)

In [ ]:
np.allclose(tf_layer,tf_layer_fd)

In [ ]:
mpl.figure(figsize=(10,10))
mpl.subplot(1,2,1)
mpl.title("TFA (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tf_layer,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.subplot(1,2,2)
mpl.title("TFA_fd (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tf_layer_fd,shape, 15)
mpl.xlabel('East y (km)')
mpl.ylabel('North x (km)')
mpl.m2km()
mpl.colorbar(pad=0.01, aspect=20, shrink=0.4).set_label('nT')
mpl.show()